# PageRank 
## Run the PageRank algorithm, modify its input, and evaluate the consequences on its output

Based on a gist by Sebastien Bratieres and on the Wikipedia page for PageRank.

PageRank was the website sorting algorithm that cemented Google's early success in the 1990s, when it displayed webpages with the highest PageRank first. Its main idea is that important webpages are likely to receive many links from other important webpages. The PageRank algorithm can be seen as computing the probability that a person who randomly clicks on links and follows them will arrive at a particular website. PageRank includes a dampening factor that represents the probability that a person will actually keep clicking as opposed to stopping where they are. It’s usually set to 0.85.

Incoming links lift a website's rank if they come from higher-ranked sites, particularly if these sites have few outgoing links. 

![PageRank illustration](http://upload.wikimedia.org/wikipedia/commons/f/fb/PageRanks-Example.svg)

In [1]:
# Encode the links present on this graph as a count matrix M_counts

import numpy as np
n_pages = 11 # numbering pages A through K as 0 to 10
M_counts = np.zeros((n_pages, n_pages)) # will hold the number of link counts (assumed 0 or 1)
# columns = starting page, row = destination page, ie M_ij = whether or not there is a link from j to i

# create variable names for each page index for clarity
pageA = 0
pageB = 1
pageC = 2
pageD = 3
pageE = 4
pageF = 5
pageG = 6
pageH = 7
pageI = 8
pageJ = 9
pageK = 10

M_counts[:,pageA] = 1 # page 0 (A in the graphic) is a sink because it has no outgoing links at all; 
# however, M cannot contain an all-zero column, so do as if A was linking to all other pages (ie put 1's everywhere)
M_counts[pageA,pageD] = 1  # to A from D
M_counts[pageB,pageC] = 1  # to B from C
M_counts[pageB,pageD] = 1  # to B from D
M_counts[pageB,pageE] = 1  # to B from E
M_counts[pageB,pageF] = 1  # to B from F
M_counts[pageB,pageG] = 1  # to B from G
M_counts[pageB,pageH] = 1  # to B from H
M_counts[pageB,pageI] = 1  # to B from I
M_counts[pageC,pageB] = 1  # to C from B
M_counts[pageD,pageE] = 1  # to D from E
M_counts[pageE,pageF] = 1  # to E from F
M_counts[pageE,pageG] = 1  # to E from G
M_counts[pageE,pageH] = 1  # to E from H
M_counts[pageE,pageI] = 1  # to E from I
M_counts[pageE,pageJ] = 1  # to E from J
M_counts[pageE,pageK] = 1  # to E from K
M_counts[pageF,pageE] = 1  # to F from E

print(M_counts)

[[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [2]:
# Convert 'M_counts' to 'M' below, by dividing each column by its sum, i.e. make sure columns sum to 1

M = np.empty((n_pages, n_pages))
for j in range(n_pages):
    M[:,j] = M_counts[:,j] / M_counts[:,j].sum()
np.set_printoptions(precision=3)
print(M)

[[0.091 0.    0.    0.5   0.    0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    1.    0.5   0.333 0.5   0.5   0.5   0.5   0.    0.   ]
 [0.091 1.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.333 0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.    0.5   0.5   0.5   0.5   1.    1.   ]
 [0.091 0.    0.    0.    0.333 0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.091 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]]


In [3]:
# Check that M has the right format to be used by the PageRank function

def check_M(M):
    n_pages = M.shape[0] # n_pages is the number of rows of M
    np.testing.assert_equal(M.shape[0], M.shape[1], err_msg = 'M should be square')
    np.testing.assert_array_almost_equal(M.sum(axis=0), np.ones((n_pages)), 
                                         err_msg = 'assert each column sums to one (M is assumed column-stochastic)')
    for j in range(n_pages):
        M_column = M[:,j]
        n_nonzero = np.count_nonzero(M[:,j])
        np.testing.assert_array_almost_equal(M_column[M_column.nonzero()], np.ones((n_nonzero)) / n_nonzero,
                                             err_msg = 'in column %g, all non-zero entries should be equal (and equal to 1 divided by their number)' % j)

check_M(M) # will produce error if M does not have the right format

In [4]:
def pagerank(M, d=0.85, square_error=1e-8):
    """
    M : the adjacency matrix of the pages. It is assumed to be column-stochastic (i.e. column sum to 1); all links have equal weight.
    A page with no outgoing links (sink) is represented as a page with outgoing links to each other page (ie restart page).
    d: damping factor (this is not in the textbook, but commonly applied in PageRank implementations; it represents the probability that a user stops clicking)
    square_error : the algorithm iterates until the difference between two successive PageRank vectors v is less than this (in squared norm)
    returns the PageRanks of all pages
    """
    n_pages = M.shape[0] # n_pages is the number of rows of M
    v = np.random.rand(n_pages) # initialize to random vector
    v = v / v.sum() # make v sum to 1
    last_v = np.ones((n_pages)) # will contain the previous v
    M_hat = d * M + (1-d)/n_pages * np.ones((n_pages, n_pages)) # this is the central PageRank equation
    while np.square(v - last_v).sum() > square_error:
        last_v = v
        v = M_hat.dot(v) # at each iteration, progress one timestep
    return v

In [5]:
result = pagerank(M)
print("Result of running PageRank on M:")
site_names = "ABCDEFGHIJK"
for i in range(10):
    print("Page", site_names[i]+":", "{:.1%}".format(result[i]))

Result of running PageRank on M:
Page A: 3.3%
Page B: 38.4%
Page C: 34.3%
Page D: 3.9%
Page E: 8.1%
Page F: 3.9%
Page G: 1.6%
Page H: 1.6%
Page I: 1.6%
Page J: 1.6%
